In [1]:
import pandas as pd
import numpy as np
def load_data():
    import pandas as pd
    import os
    df = pd.read_csv("../model/total_data.csv")

    y = df["Phase"]
    x = df.drop(columns=["Phase"])
    return x, y

def make_train_test_split(x, y):

    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler

    (x_train, x_test, y_train, y_test) = train_test_split(
        x,
        y,
        test_size=0.25,
        random_state=123456,
    )

    X_train = x_train.apply(pd.to_numeric, errors='coerce')
    X_test = x_test.apply(pd.to_numeric, errors='coerce')
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_train_scaled = pd.DataFrame(X_train_scaled).fillna(0)
    X_test_scaled = pd.DataFrame(X_test_scaled).fillna(0)


    return X_train_scaled, X_test_scaled, y_train, y_test

def eval_metrics(y_true, y_pred):

    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    return mse, mae, r2

def report(estimator, mse, mae, r2):

    print(estimator, ":", sep="")
    print(f"  MSE: {mse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")

def set_tracking_uri():

    import mlflow

    mlflow.set_tracking_uri('sqlite:///mlruns.db')


def run():
    #
    # Entrena un modelo sklearn ElasticNet
    #
    import sys
    from sklearn.ensemble import RandomForestClassifier
    import mlflow


    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)

    n_estimators = 100
    max_depth = 10
    verbose = 1

    set_tracking_uri()
    print('Tracking directory:', mlflow.get_tracking_uri())

    with mlflow.start_run():
        estimator = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        estimator.fit(x_train, y_train)
        mse, mae, r2 = eval_metrics(y_test, y_pred=estimator.predict(x_test))
        if verbose > 0:
            report(estimator, mse, mae, r2)

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        mlflow.log_metric("mse", mse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)

        mlflow.sklearn.log_model(estimator, "model")

if __name__ == "__main__":
    run()

/tmp/ipykernel_1278/2461984980.py:6: DtypeWarning: Columns (106,148,171,178) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../model/total_data.csv")
/usr/local/lib/python3.8/dist-packages/sklearn/utils/extmath.py:980: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.8/dist-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.8/dist-packages/sklearn/utils/extmath.py:1005: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Tracking directory: sqlite:///mlruns.db


2023/03/26 17:50:09 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/03/26 17:50:09 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

RandomForestClassifier(max_depth=10, random_state=42):
  MSE: 0.17144638403990026
  MAE: 0.1701995012468828
  R2: 0.5961669274354724


In [25]:
def list_models():

    from pprint import pprint

    import mlflow

    client = mlflow.tracking.MlflowClient()

    for rm in client.search_registered_models():
        pprint(dict(rm), indent=4)


list_models()

{   'creation_timestamp': 1679857486643,
    'description': '',
    'last_updated_timestamp': 1679857486811,
    'latest_versions': [   <ModelVersion: creation_timestamp=1679857486811, current_stage='None', description='', last_updated_timestamp=1679857486811, name='Phase', run_id='e3b227c5bf82477bafb5a95ecb3408d4', run_link='', source='./mlruns/0/e3b227c5bf82477bafb5a95ecb3408d4/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>],
    'name': 'Phase',
    'tags': {}}


In [24]:
def predict():

    import mlflow

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)

    model_name = "Phase"
    model_version = 1

    model = mlflow.pyfunc.load_model(
        model_uri=f"models:/{model_name}/{model_version}"
    )
    print(x_test[0:10])

    return model.predict(x_test[0:10])


predict()

/tmp/ipykernel_1278/2461984980.py:6: DtypeWarning: Columns (106,148,171,178) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../model/total_data.csv")
/usr/local/lib/python3.8/dist-packages/sklearn/utils/extmath.py:980: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.8/dist-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.8/dist-packages/sklearn/utils/extmath.py:1005: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


        0         1         2         3         4         5         6    \
0 -1.067457  1.403464 -0.317637 -0.308081 -0.213929 -0.140410 -0.067125   
1 -1.319033  1.617971 -0.274978 -0.308081 -0.213929 -0.140410 -0.067125   
2  0.406058 -0.884615  0.663530 -0.308081 -0.213929 -0.140410 -0.067125   
3  0.298240 -0.062337 -0.744231  0.617556  2.856739  0.713023 -0.067125   
4 -0.815881  1.653723 -0.914869 -0.308081 -0.213929 -0.140410 -0.067125   
5 -0.851820 -0.616481  1.836664 -0.308081 -0.213929 -0.140410 -0.067125   
6  0.765452 -0.705859  0.023638 -0.308081 -0.213929 -0.140410 -0.067125   
7 -1.031517  1.081703  0.023638 -0.308081 -0.213929 -0.140410 -0.067125   
8  1.196725 -0.598605 -0.616253 -0.308081 -0.213929 -0.140410 -0.067125   
9  1.663937 -0.705859 -1.042847 -0.308081 -0.213929 -0.140410 -0.067125   

        7         8         9    ...  186  187  188  189  190  191  192  193  \
0 -0.033874  0.210314  0.323755  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1 -0.033874 -0

array([1, 1, 1, 0, 1, 1, 2, 1, 1, 1])

In [18]:
def get_json_test_data():

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)

    data = x_test.iloc[0:1,:].to_json(orient='split')

    data = repr(data)
    return data

data = get_json_test_data()
data

#data=json.loads(data)

#data["data"]

/tmp/ipykernel_1278/2461984980.py:6: DtypeWarning: Columns (106,148,171,178) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../model/total_data.csv")
/usr/local/lib/python3.8/dist-packages/sklearn/utils/extmath.py:980: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.8/dist-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.8/dist-packages/sklearn/utils/extmath.py:1005: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


'\'{"columns":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195],"index":[0],"data":[[-1.067456824,1.4034639165,-0.3176370173,-0.3080809156,-0.2139287125,-0.1404102625,-0.0671249567,-0.0338743355,0.2103138093,0.3237545679,0.0,0.0887977933,2.6983516577,0.3713863092,-0.0141246184,0.1183396728,-0.3584991432,0.5337279303,1.2231844741,0.4447388115,-0.9579553629,1.1151428139,0.2

In [34]:
import json
!curl http://127.0.0.1:5000/invocations -H 'Content-Type: application/json' {data}
#'{"dataframe_records": [{"0": -0.4060584017916565, "1": 0.88461498872638, "2": 0.663529993511018, "3": 0.30808091564938983, "4": 0.21392871253652665, "5": 0.14041026250531424, "6": 0.0671249566589358, "7": 0.03387433551322876, "8": 1.955812034441059, "9": 0.14252177885316064, "10": 0.6298723128021928, "11": 1.0568627044895678, "12": 0.5576905325681901, "13": 0.5288250640567684, "14": 1.109127422090444, "15": 0.5972798603847551, "16": 2.215339445463756, "17": 0.3239090345306585, "18": 0.9843816317006839, "19": 0.4447388115023369, "20": 0.9579553629029113, "21": 0.7036611643582424, "22": 1.4453040554742085, "23": 0.027974969980362065, "24": 0.6129013285819107, "25": 0.020658246771404105, "26": 1.4349215081455242, "27": 0.9432278308077526, "28": 0.6868625213705105, "29": 3.2601180753168966, "30": 2.369816060632318, "31": 0.0, "32": 2.3820054957286514, "33": 1.7405485553475915, "34": 0.0, "35": 0.7244343106799591, "36": 0.3358688879269378, "37": 1.9471000820518918, "38": 1.1860397413441577, "39": 2.016888872814617, "40": 1.34278937548285, "41": 1.868509475297495, "42": 1.1691786550694092, "43": 0.4944399499732391, "44": 0.3675799735871101, "45": 1.0158635705078347, "46": 1.7241148974492486, "47": 0.5758581145390598, "48": 0.8142303147245551, "49": 0.8852950396914213, "50": 2.0656432618741523, "51": 0.025759284550112352, "52": 0.6394741321322247, "53": 1.4426034884782428, "54": 0.09037341676144327, "55": 0.8590425112549828, "56": 0.1896591266922299, "57": 0.0, "58": 0.03864406257389803, "59": 1.676774864799105, "60": 0.19191670192940216, "61": 0.59065283428342, "62": 0.6981192246771619, "63": 0.49568104564430165, "64": 0.24015323539955838, "65": 1.352910695930345, "66": 0.369128655652966, "67": 0.4592261665997708, "68": 1.239694379192309, "69": 0.6186018475558498, "70": 0.5006518473807843, "71": 0.5186924547915595, "72": 0.021100447837508904, "73": 1.1492845104861547, "74": 0.6747212705023411, "75": 0.1367586140517963, "76": 0.5958550590211021, "77": 0.16677329945024613, "78": 0.0, "79": 0.0, "80": 0.0, "81": 0.0, "82": 0.0, "83": 0.0, "84": 0.0, "85": 0.0, "86": 0.0, "87": 0.0, "88": 0.0, "89": 0.0, "90": 0.0, "91": 0.0, "92": 0.0, "93": 0.0, "94": 0.0, "95": 0.0, "96": 0.0, "97": 0.0, "98": 0.0, "99": 0.0, "100": 0.0, "101": 0.0, "102": 0.0, "103": 0.0, "104": 0.0, "105": 0.0, "106": 0.0, "107": 0.0, "108": 0.0, "109": 0.0, "110": 0.0, "111": 0.0, "112": 0.0, "113": 0.0, "114": 0.0, "115": 0.0, "116": 0.0, "117": 0.0, "118": 0.0, "119": 0.0, "120": 0.0, "121": 0.0, "122": 0.0, "123": 0.0, "124": 0.0, "125": 0.0, "126": 0.0, "127": 0.0, "128": 0.0, "129": 0.0, "130": 0.0, "131": 0.0, "132": 0.0, "133": 0.0, "134": 0.0, "135": 0.0, "136": 0.0, "137": 0.0, "138": 0.0, "139": 0.0, "140": 0.0, "141": 0.0, "142": 0.0, "143": 0.0, "144": 0.0, "145": 0.0, "146": 0.0, "147": 0.0, "148": 0.0, "149": 0.0, "150": 0.0, "151": 0.0, "152": 0.0, "153": 0.0, "154": 0.0, "155": 0.0, "156": 0.0, "157": 0.0, "158": 0.0, "159": 0.0, "160": 0.0, "161": 0.0, "162": 0.0, "163": 0.0, "164": 0.0, "165": 0.0, "166": 0.0, "167": 0.0, "168": 0.0, "169": 0.0, "170": 0.0, "171": 0.0, "172": 0.0, "173": 0.0, "174": 0.0, "175": 0.0, "176": 0.0, "177": 0.0, "178": 0.0, "179": 0.0, "180": 0.0, "181": 0.0, "182": 0.0, "183": 0.0, "184": 0.0, "185": 0.0, "186": 0.0, "187": 0.0, "188": 0.0, "189": 0.0, "190": 0.0, "191": 0.0, "192": 0.0, "193": 0.0, "194": 0.0, "195": 0.0}]}'



<!doctype html>
<html lang=en>
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>
curl: (3) nested brace in URL position 12:
{"columns":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,
